# 传家之宝 - 模型训练实验 Notebook

本 Notebook 用于手动测试和调试训练流程，包括：
- JSONL 数据生成
- System Prompt 定制
- 模型训练流程
- 推理测试

In [ ]:
# 导入必要的模块
import sys
sys.path.append('..')

from config.config_loader import config
from utils.logger import logger
from utils.jsonl_builder import JSONLBuilder
from utils.system_prompt import SystemPromptGenerator
from core.trainer import OllamaTrainer
from core.model_manager import ModelManager
from core.progress_tracker import ProgressTracker

import json
import pandas as pd

## 1. 查看当前配置

In [ ]:
# 查看训练配置
print("=== 训练配置 ===")
training_config = config.get_training_config()
for key, value in training_config.items():
    print(f"{key}: {value}")

print("\n=== 当前基础模型 ===")
current_model = config.get_current_model()
for key, value in current_model.items():
    print(f"{key}: {value}")

## 2. 测试 JSONL 数据生成

In [ ]:
# 设置测试老人 ID 和姓名
TEST_ELDER_ID = "LXM19580312M"
TEST_ELDER_NAME = "李小明"

# 创建 JSONL 构建器
builder = JSONLBuilder()

# 连接数据库并拉取记忆数据
with builder:
    memories = builder.fetch_elder_memories(TEST_ELDER_ID)
    print(f"找到 {len(memories)} 条记忆数据")
    
    # 显示前几条记忆
    if memories:
        print("\n=== 前 3 条记忆示例 ===")
        for i, memory in enumerate(memories[:3], 1):
            print(f"\n记忆 {i}:")
            print(f"问题: {memory['question'][:50]}...")
            print(f"回答: {memory['answer'][:100]}...")
            print(f"分类: {memory['category']}")

In [ ]:
# 生成 JSONL 文件
jsonl_path = builder.build_jsonl(TEST_ELDER_ID, elder_name=TEST_ELDER_NAME)
print(f"JSONL 文件已生成: {jsonl_path}")

# 读取并查看前几行
if jsonl_path:
    print("\n=== JSONL 内容示例 ===")
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 2:  # 只显示前 2 条
                break
            data = json.loads(line)
            print(f"\n样本 {i+1}:")
            print(data['text'][:200] + "...")

## 3. 测试 System Prompt 生成

In [ ]:
# 创建 System Prompt 生成器
prompt_gen = SystemPromptGenerator()

# 生成基础 prompt
basic_prompt = prompt_gen.generate(TEST_ELDER_NAME)
print("=== 基础 System Prompt ===")
print(basic_prompt)

# 生成带记忆示例的 prompt
if memories:
    memory_prompt = prompt_gen.generate(TEST_ELDER_NAME, memories[:3])
    print("\n=== 带记忆示例的 System Prompt ===")
    print(memory_prompt)

## 4. 测试模型管理器

In [ ]:
# 创建模型管理器
manager = ModelManager()

# 列出所有 AFS 模型
models = manager.list_models(filter_afs_only=True)
print(f"=== 找到 {len(models)} 个 AFS 模型 ===")
for model in models:
    print(f"- {model['name']} ({model['size']})")

# 检查测试模型是否存在
exists = manager.model_exists(TEST_ELDER_ID)
print(f"\n模型 {TEST_ELDER_ID} 是否存在: {exists}")

## 5. 测试训练流程（可选 - 需要实际数据）

In [ ]:
# 创建训练器
trainer = OllamaTrainer()

# 检查 Ollama 是否可用
ollama_available = trainer.check_ollama_available()
print(f"Ollama 可用: {ollama_available}")

if ollama_available:
    print("\n可以开始训练！")
    print("注意：训练可能需要较长时间，请在确保数据准备完成后再执行。")
    
    # 取消下面注释以实际执行训练
    # result = trainer.train(TEST_ELDER_ID, TEST_ELDER_NAME)
    # print(f"\n训练结果: {result}")
else:
    print("\n⚠️ Ollama 不可用，无法进行训练")

## 6. 测试进度跟踪器

In [ ]:
# 创建进度跟踪器
tracker = ProgressTracker()

# 模拟训练进度
import time

job_id = tracker.start_tracking(TEST_ELDER_ID, total_epochs=3)
print(f"创建训练任务: {job_id}")

for epoch in range(1, 4):
    time.sleep(1)
    tracker.update_progress(job_id, status='training', current_epoch=epoch)
    tracker.add_log(job_id, f"正在训练 Epoch {epoch}/3")
    
    progress = tracker.get_progress(job_id)
    print(f"Epoch {epoch}: 进度 {progress['progress']}%, ETA: {progress.get('eta', 'N/A')}")

tracker.complete_tracking(job_id, success=True)
print("\n训练模拟完成！")

## 7. 数据分析（可选）

In [ ]:
# 将记忆数据转换为 DataFrame 进行分析
if memories:
    df = pd.DataFrame(memories)
    
    print("=== 数据统计 ===")
    print(f"总记忆数: {len(df)}")
    print(f"\n分类分布:")
    print(df['category'].value_counts())
    
    print(f"\n问题长度统计:")
    df['question_length'] = df['question'].str.len()
    print(df['question_length'].describe())
    
    print(f"\n回答长度统计:")
    df['answer_length'] = df['answer'].str.len()
    print(df['answer_length'].describe())

## 总结

本 Notebook 展示了：
1. ✅ 配置加载和验证
2. ✅ JSONL 数据集生成
3. ✅ System Prompt 定制
4. ✅ 模型管理功能
5. ✅ 训练流程框架
6. ✅ 进度跟踪功能
7. ✅ 数据分析工具

你可以根据实际需求修改参数和测试不同场景。